Nous allons détailler les fonctions dans ce notebook.

### IMPORTS

In [38]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from pathlib import Path
import os
# from webdriver_manager.chrome import ChromeDriverManager
# import chromedriver_binary
import re
import csv

### URLs exemple

In [39]:
url_kindle_accueil = "https://www.amazon.com/b?node=154606011"
url_kindle_cat_art_and_photo ="https://www.amazon.com/s?bbn=154606011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011&dc&qid=1648440703&rnid=154606011&ref=lp_154606011_nr_n_0"
url_kindle_cat_art_and_photo_subcat_music_p1 = "https://www.amazon.com/s?rh=n%3A156302011&fs=true&ref=lp_156302011_sar"

url_kindle_cat_art_and_photo_subcat_umusic_p2 = "https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=2&qid=1648440846&ref=sr_pg_2"
url_kindle_cat_art_and_photo_subcat_umusic_p3  ="https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=3&qid=1648440968&ref=sr_pg_3"

In [40]:
url_livre1="https://www.amazon.com/Last-Goddess-Novel-Kate%C5%99ina-Tu%C4%8Dkov%C3%A1-ebook/dp/B09BZ8L74Q/ref=pd_sbs_sccl_1_3/131-3814768-2928133?pd_rd_w=eOyFv&pf_rd_p=3676f086-9496-4fd7-8490-77cf7f43f846&pf_rd_r=XXEVJ89KR0WC7CQF9791&pd_rd_r=ff1f8e44-e48e-4dcc-8716-a64d5675ebe6&pd_rd_wg=tAIJl&pd_rd_i=B09BZ8L74Q&psc=1"
url_livre2="https://www.amazon.com/gp/product/B09K7J47Y8/ref=s9_acss_bw_cg_bsmafr_1b1_w?pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-13&pf_rd_r=EX2QJTNDW0DCKTRX8KFT&pf_rd_t=101&pf_rd_p=5074797f-6c32-4283-a5a7-526fd1198645&pf_rd_i=14413390011"
url_livre3="https://www.amazon.com/Healthy-Smoothie-Cookbook-Breakfast-Kid-Friendly-ebook/dp/B083QK3DT9/ref=sr_1_12?keywords=smoothie+recipe+book&qid=1651700938&s=digital-text&sprefix=smoothie%2Cdigital-text%2C236&sr=1-12"

# Ouverture du navigateur

avant bien lancer le programme X launch sur windows en cochant sur disable

In [41]:
DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve())
browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)
browser.get(url_kindle_accueil)

/tmp/ipykernel_878/2347370238.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)


avec le programme ci-après j arrive à lancer chrome depuis notebook !! yes

>### il faut réussir à faire différents programme:
>   - sur la page d'un livre ( en input un url) trouver le livre , l'auteur, le ranking
>   - sur une page, trouver tout les livres de la pages
>   - sur une catégorie trouver les pages de catégories

# Fonction 1 : avec url d'un livre , retourner ranking des ventes, le nom l'auteur 

In [42]:
regex_expression = "</span> #.* in Kindle Store " # utile pour regex

In [43]:
url_yumkasi = "https://www.amazon.com/Minoru-Yamasaki-Humanist-Architecture-Modernist-ebook/dp/B0798MHV9R/ref=sr_1_17?qid=1652127157&rnid=157630011&s=digital-text&sr=1-17"

In [44]:
url_livre ="https://www.amazon.com/gp/product/B09XFL8N77?storeType=ebooks&pf_rd_p=1a003f54-b0fb-4c1f-847d-4b886d085c2a&pf_rd_r=1S6X1FD1D0CDDESY6F9C&pd_rd_wg=OYcWw&pd_rd_i=B09XFL8N77&ref_=dbs_0_def_rwt_wigo_nfy_recs_wigo_1&pd_rd_w=n45p7&pd_rd_r=29980150-6069-4b0c-9b26-a02de52121aa"
browser.get(url_livre)

In [45]:
browser.get(url_yumkasi)

In [46]:
code_de_la_page = browser.page_source
soup = BeautifulSoup(code_de_la_page , 'lxml')

In [47]:
titre_livre = soup.find('span'  , {'class': 'a-size-extra-large'}).string

In [48]:
titre_livre

' Minoru Yamasaki: Humanist Architecture for a Modernist World '

In [49]:
rang_livre = soup.find_all( {'span'  : 'a-list-item'})

In [50]:
rang_vente = str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",","")

In [51]:
str(rang_vente)

'1169342'

In [52]:
def trouver_rang(url):
    browser.get(url)
    code_de_la_page = browser.page_source
    soup = BeautifulSoup(code_de_la_page , 'lxml')
    rang_livre = soup.find_all( {'span'  : 'a-list-item'})
    rang_vente = str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",","")
    return str(rang_vente)

In [53]:
trouver_rang(url_livre2)

'14'

In [54]:
books_html = soup.find_all("article", class_="product_pod")
len(books_html)

0

# Sur une page récupérer les données : de classement de vente , nom du livre catérgorie

In [55]:
url = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&qid=1651987607&rnid=157630011&ref=sr_nr_n_1"
url_page_1 = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=1&qid=1651987613&rnid=157630011&ref=sr_pg_2"
url_page_2 = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=2&qid=1651987613&rnid=157630011&ref=sr_pg_2"
url_page_3     = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=3&qid=1651987613&rnid=157630011&ref=sr_pg_3"
url_page_3bis  = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=3&qid=1651987793&rnid=157630011&ref=sr_pg_3"


In [56]:
browser.get(url_page_2) #navigue sur une page avec tous les lien du site
html_page_listing = browser.page_source # permet de prendre le code html de la page 
soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} )
#tous les livres de la pages

# le numéro ASIN correspeond à la ref du livre sur amazon
# ASIN=B098652LH8P

In [57]:
len(livres)

16

In [58]:
url_du_livre = 'https://www.amazon.com'+livres[2].h2.a.get('href')

In [59]:
url_du_livre

'https://www.amazon.com/Higher-Historic-Race-Making-City-ebook/dp/B000FBJCQW/ref=sr_1_19?qid=1652243416&rnid=157630011&s=digital-text&sr=1-19'

In [60]:
browser.get(url_du_livre)

In [61]:
trouver_rang("https://www.amazon.com/Minoru-Yamasaki-Humanist-Architecture-Modernist-ebook/dp/B0798MHV9R/ref=sr_1_17?qid=1652130424&rnid=157630011&s=digital-text&sr=1-17")

'1169342'

In [103]:
def ecrire_csv(ads):
    with open('results.csv','a') as f :
        fields=['title','price','url','rang']
        writer = csv.DictWriter(f,fieldnames = fields)
        for ad in ads :
            print(ad)
            writer.writerows(ads)

In [109]:
liste_test=[{'title':'titre1','price':'18euros','url':'www.lelirvr.com','rang':'1874'},{'title':'titre2','price':'19euros','url':'www.lelirvr2.com','rang':'2222'}]

In [110]:
ecrire_csv(liste_test)#permet d'écire uen liste de dictionnaire

In [108]:
for livre in livres :
    h2 = livre.h2
    titre = h2.text.strip()
    url_du_livre = 'https://www.amazon.com' + h2.a.get('href')
    prix = livre.find('span' , class_ = 'a-price-whole').text.strip(' ').strip('.')
    rang = trouver_rang(url_du_livre)
    #print(titre,url_du_livre,rang,prix,'€')
    liste_in = [titre,prix,url_du_livre,rang]
    ecrire_csv(liste_in)

AttributeError: 'str' object has no attribute 'keys'